In [ ]:
!pip install pandas
!pip install -U scikit-learn
!pip install graphviz

In [ ]:
import pandas as pd

uri = 'https://gist.githubusercontent.com/guilhermesilveira/4d1d4a16ccbf6ea4e0a64a38a24ec884/raw/afd05cb0c796d18f3f5a6537053ded308ba94bf7/car-prices.csv'
dados = pd.read_csv(uri)
dicionario = {
    'mileage_per_year': 'milhas_ano',
    'model_year': 'ano_modelo',
    'price': 'preco',
    'sold': 'vendido'
}

dados = dados.rename(columns = dicionario)
dados.vendido = dados.vendido.map({ 'yes': 1, 'no': 0})
dados.head()

In [ ]:
from datetime import datetime

ano_atual = datetime.today().year
dados['idade_modelo'] = ano_atual - dados['ano_modelo']
dados.head()

In [ ]:
dados['km_ano'] = dados['milhas_ano'] * 1.60934
dados.head()

In [ ]:
dados = dados.drop(columns = ['Unnamed: 0', 'milhas_ano', 'ano_modelo'], axis=1)
dados.head()

# Rodando LinearSVC

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
import numpy as np

x = dados[['preco', 'idade_modelo', 'km_ano']]
y = dados['vendido']

SEED = 5
np.random.seed(SEED)
treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, stratify = y, test_size = 0.25)
print('Treinaremos com %d elementos e testaremos com %d elementos' % (len(treino_x), len(teste_x)))

modelo = LinearSVC()
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)
acuracia = accuracy_score(teste_y, previsoes) * 100

print("A acurácia foi %.2f%%" % acuracia)

In [ ]:
from sklearn.dummy import DummyClassifier
dummy_stratified = DummyClassifier()
dummy_stratified.fit(treino_x, treino_y)
acuracia = dummy_stratified.score(teste_x, teste_y) * 100

print("A acurácia do dummy_stratified foi %.2f%%" % acuracia)

In [ ]:
from sklearn.dummy import DummyClassifier
dummy_mostfrequent = DummyClassifier()
dummy_mostfrequent.fit(treino_x, treino_y)
acuracia = dummy_mostfrequent.score(teste_x, teste_y) * 100

print("A acurácia do dummy_mostfrequent foi %.2f%%" % acuracia)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

SEED = 5
np.random.seed(SEED)
raw_treino_x, raw_teste_x, treino_y, teste_y = train_test_split(x, y, test_size = 0.25,
                                                         stratify = y)
print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(treino_x), len(teste_x)))

scaler = StandardScaler()
scaler.fit(raw_treino_x)
treino_x = scaler.transform(raw_treino_x)
teste_x = scaler.transform(raw_teste_x)

modelo = SVC()
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes) * 100
print("A acurácia foi %.2f%%" % acuracia)

Utilizando a scaller há muito ganho de acurácia

# Árvores de decisão

## Árvores de decisão não precisam do escalador para funcionar bem.

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

SEED = 5
np.random.seed(SEED)
raw_treino_x, raw_teste_x, treino_y, teste_y = train_test_split(x, y, test_size = 0.25, stratify = y)
print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(treino_x), len(teste_x)))

modelo = DecisionTreeClassifier(max_depth=3)
modelo.fit(raw_treino_x, treino_y)
previsoes = modelo.predict(raw_teste_x)

acuracia = accuracy_score(teste_y, previsoes) * 100
print("A acurácia foi %.2f%%" % acuracia)

In [ ]:
from sklearn.tree import export_graphviz
import graphviz

features = x.columns
dot_data = export_graphviz(
    modelo, out_file=None, feature_names = features, 
    filled = True, rounded = True, class_names = ['não', 'sim'],
    
)
grafico = graphviz.Source(dot_data)
grafico